# Introductory workshop Machine Learning: Data cleaning
Welcome to this introductory workshop to machine learning.






During this workshop you will learn:
 - How to explore your data 
 - How to clean your data 
 - How to create a machine learning model to predict if a bus is delayed, on time.
 
 
 
 Raw dataset directly from Kolumbus. Not processed in any way
 
 
You will work with the following data:


# The data
![https://www.kolumbus.no/ruter/kart/sanntidskart/](media/sanntidskart.png) https://www.kolumbus.no/ruter/kart/sanntidskart/

The data you will be working with in this workshop is real time bus data from the [Kolumbus](https://en.wikipedia.org/wiki/Kolumbus), the public transportation administration in Rogaland, Norway. 

Kolumbus visualizes this data in their real map: https://www.kolumbus.no/ruter/kart/sanntidskart/ *(As of june 2018, the delay is not visualized in the real time map as it is in the screenshot above. The buses should changed color depending on the delay)*


Every time a bus moves results in one new row of data.  Each row consists of the buses position, schedules, route information and the degree of delay.



## Columns
| Name                        	| Description                                                                                                                                                                                	| Example                         	|
|-----------------------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|----------------------------------	|
| Delay                       	| The total delay in seconds. Denotes the number of seconds behind the current schedule. A negative delay means that the bus is ahead of schedule                                                                  	| 60                                	|
| Line                        	| Line is the human readable name of the current line. Oslo-Stavanger and Stavanger-Oslo can both be Line 7                                                                       	| 8                                	|
| LineID                      	| LineID identies a line. Oslo-Stavanger og Stavanger-Oslo have different LineIDs but the same Line. Take not that two buses with same LineId will may necessarily drive the same physical route                        	| 3022                             	|
| TripID                      	| TripID detones a spesific departue. Oslo-Stavanger leaving at 15:00 has a unique TripID but will have the same LineID as Oslo-Stavanger 16:00. Each TripID is only used once a day.                           	| 30222006                         	|
| DirectionRef                	| Binary variable eaither set to ’go’ or ’back’. Oslo – Stavanger vs. Stavanger – Oslo                                                                                         	| 'go'                             	|
| TripHeadsign                	| General area where the bus is headed. This the inforamtion that is displayed at the front of the bus                                                                                                                                                               	| 'Tasta - Vardeneset – Randaberg' 	|
| DestinationAimedArrivalTime 	| When the bus is supposed to arrive at the final destination                                                                                                                                	| 2017-03-06 00:19:00              	|
| DestinationTerminalCode     	| Id for the final stop of the trip.                             	| 11275943                         	|
| To                          	| Where the bus is headed. Not necessarily the name of the first stop                                                                                                                       	| 'Randaberg sentrum'              	|
| OriginAimedDepartureTime    	| When the bus is supposed to depart from the first station                                                                                                                                                       	| 2017-03-05 23:44:00              	|
| OriginStopTerminalCode      	| Id for the first stop                                                                                                                                                                    	| 11034315                         	|
| From                        	| The start location of the bus. Not necessarily the name of the first stop                                                                                                                           	| 'Stavanger '              	|
| Position                   	| The position stored as key value pair                                                                                                                                      	| {'lat': 63.36706, 'lng': 10.37173}                             	|
| VehicleModes                        	| The kind of vehicle. Bus, boat, plane?                                                                                                                    	| [1]              	|
| DistanceBetweenStops        	| Distance measured in meters to the next stop. Take into account the actuall road the bus has to travel                                                                         	| 20.0                             	|
| PercentageBetweenStops      	| How far the bus has travel on the path between the last stop and the next stop.  	| 5.0                              	|
| Heading                     	| Rotation of the bus in degrees. heading=0 is facing north                                                                                                                                                 	| -80.2347288919                   	|
| IsMonitored                     	| Is the bus monitored by the real itme system?                                                                                                                                                  	| True                  	|
| Latitude                    	| Coordinates                                                                                                                                                                            	| 58.98848                         	|
| Longitude                   	| Coordinates                                                                                                                                                                            	| 5.673485                         	|
| NextStop                    	| Human readable name of the next stop. Multiple stops may share the same name                                                                                                        	| 'Eskelandstunet'                 	|
| NextStopCode                	| Each bus stop has its own Id. Two stops at each side of the road may have the same name (column NextStop), but will alwasy have an unqie NextStopCode.                                          	| 11031438                         	|
| RecordedAtTime              	| Date and time when the row was recorded         	| 2017-03-10 23:59:39              	|
| Id                   	| Unique ID for each physical bus                                                                                                                                       	| 2001                             	|


# Read Data

The SIRI system writes one JSON file about every second. This file contains the positopn of every bus at Kolumbus disposale. As we will inspect shortly, most these buses are not active.

For this workshop, we have sampled 263 such JSON files distriputed througout 11. March 2017.

The Python code below reads these JSON files and assembles them to a single Pandas Dataframe for further processing. 
    * Open "Running Code.ipynb" if you are unfamiliar with Jupyter Notebooks and how to run code
    
The first thing we need to do is to unzip the data.

In [ ]:
import zipfile
from pathlib import Path

my_file = Path("data/random_one_day.zip")
if my_file.is_file():
    with zipfile.ZipFile("data/random_one_day.zip","r") as zip_ref:
        zip_ref.extractall("data")
    print("Done extracting the data!")
else:
    print("You need to download")

In [ ]:
import read_json
import pandas as pd
from test_dataCleaner import TestDataCleaner
cleaner = TestDataCleaner()
frame = read_json.get_dataframe()

print("Number of rows:",len(frame))

# Inspecting the data
With the files imported, we can inspect the data.

Below is a snippet of the raw data recived from Kolumbus. 

In [ ]:
frame.head(3) # Displays the first 3 lines of data

As you can see, the data contains a great deal of missing values, NaN.

# Dealing with empty fields
Most of the real time data from Kolumbus have missing route information. 

Buses that are not in active service continue to transmit their location. These are not shown on the real time map

Another issue is that the bus driver has to manually enter which route he or she is driving. This leads to problems where active buses are not tied to a route or tied to the the wrong route. These buses are displayed on the [Kolumbus real time map](https://www.kolumbus.no/ruter/kart/sanntidskart/) in grey


We only want the buses that are activly driving on a route. The first task is to remove all rows with missing route information. 


In [ ]:
columns = [] # The columns listed here are required to have a value or the entire row is removed columns = ['column_1','column_2','column_2']
columns = ['DestinationAimedArrivalTime','OriginAimedDepartureTime','Delay','TripId'] # CORRECT ANSWER, TO BE REMOVED

test_frame = frame.dropna(subset=columns)
print("Removed",len(frame)-len(test_frame),"rows with missing values")

cleaner.remove_null_rows(test_frame)

In [ ]:
frame = test_frame # Do not run this cell until the test is passed.

# Inspecting the data II
With most of the missing values now removed, we can take a better look at the data and all the available fields.

In [ ]:
frame.head(3) # Displays the first 3 lines of data

In [ ]:
frame.tail(3) # Displayes that last 3 rows

In [ ]:
frame[['Longitude','Latitude']].describe() # Use describe() to show usefull statstics about given columns. HINT: The max and min values are especially useful later on..

# Duplicates

Study the the updates from the bus line 5 below. Notice any similarities between the rows?


In [ ]:
frame[frame['TripId'] == '10061098'].head(3) 

The real time transit system is updated about every second. The buses however, only updates their position a couple of times a minute. The data we recived form Kolumbus therfore contains a great deal of duplicates as the previous update is repteated until a new one is recived. 


Removes every duplicated row where every value in the specified columns are the same

            What might be used to identify a duplicated row? Ask yourself:
            * Can there be several messages sent at the same time from the same bus?
            * Can buses leave or arrive at the same time?
            * Could they be at the same place at the same time?
            * How specific do you need to be with line, lineId or TripID?

In [ ]:
columns = ['column_1', 'column_2', 'column_3'] # Removes all duplicate rows were 'column_1', 'column_2', 'column_3' have identical values. columns = ['column_1', 'column_2', 'column_3'] 
columns = ['RecordedAtTime', 'TripId'] # CORRECT ANSWER, TO BE REMOVED

test_frame = frame.drop_duplicates(columns)

print("Removed",len(frame)-len(test_frame),"duplicated rows")
cleaner.remove_duplicate_entries(test_frame)

In [ ]:
frame = test_frame

# Drop unwanted columns
In this section, your task is to reduce the size of the data by removing columns we dont need. 

Below are some tools you should use to familiarize yourself more with the data

In [ ]:
frame.head(10)

In [ ]:
frame[['Delay']].describe(include='all')

In [ ]:
frame['Line'].value_counts().head(20) # Lists every unique value for the given column and frequencies. .head(20) limits the list to 20 most frequently used

### Columns with repeating value

As you might have noticed, some of the columns in the Kolumbus data have a single repating value for all rows. Every row is identical and therfor no use for use. 

Find these columns and remove them.

In [ ]:
columns = [] # Removes the columns listed here. columns = ['column_1', 'column_2', 'column_3']
columns = ['NextStopVisitNumber', 'VehicleModes'] # CORRECT ANSWER, TO BE REMOVED

test_frame = frame.drop(columns,1)
cleaner.remove_hard_coded_columns(test_frame)

In [ ]:
frame = test_frame

### Columns with missing rows

Some of the columns does still contain rows with missing data (NaN). There is not one general solution on how to deal with such data. In some cases, the missing values can be raplaced with a known value or perhaps the previous ones. In this case, these columns are not important and should be removed.

In [ ]:
columns = [] # Removes the columns listed here. 
columns = ['Heading','IsMonitored','TripHeadsign'] # CORRECT ANSWER, TO BE REMOVED

test_frame = frame.drop(columns,1) # CORRECT ANSWER, TO BE REMOVED
cleaner.remove_columns_with_null_values(test_frame)

In [ ]:
frame = test_frame

### Redundant columns

Multiple columns might contain similar or even identical information This is also the case for this dataset. 

Use the tools. Keep the columns that are most similar to the rest of the data as this is the most convinient.

In [ ]:
columns = [] # Removes all duplicate rows were 'column_1', 'column_2', 'column_3' have identical values
#columns = ['Position'] # CORRECT ANSWER, TO BE REMOVED

test_frame = frame.drop(columns,1)
cleaner.remove_redundant_columns(test_frame)

In [ ]:
frame = test_frame



# Outliers

From [Wikipedia](https://en.wikipedia.org/wiki/Outlier): "In statistics, an outlier is an observation point that is distant from other observations"

Removes outliers and data missing certain values. Poor data quality as input results in poor predictions.


In [ ]:
from datetime import datetime
frame['RecordedAtTime'] = frame['RecordedAtTime'].astype('datetime64[ns]')
frame['OriginAimedDepartureTime'] = frame['OriginAimedDepartureTime'].astype('datetime64[ns]')
frame['DestinationAimedArrivalTime'] = frame['DestinationAimedArrivalTime'].astype('datetime64[ns]')
test_frame = frame

### Time

You have data from 10. March 2017, or do you?
Of the three columns related to time, are there some event that should always occur before others?

In [ ]:
test_frame = test_frame[(test_frame['RecordedAtTime'] > datetime(1900, 1, 1))]
test_frame = test_frame[(test_frame['OriginAimedDepartureTime'] < test_frame['DestinationAimedArrivalTime'])] # This should always be true, a bus cannot arrive before it departs. 
# Are the more similar scenarios related to time? 

test_frame = test_frame[(test_frame['RecordedAtTime'] > datetime(2017, 3, 1))] # CORRECT ANSWER, TO BE REMOVED
test_frame = test_frame[(test_frame['RecordedAtTime'] < datetime(2017, 3, 20))] # CORRECT ANSWER, TO BE REMOVED
test_frame = test_frame[(test_frame['OriginAimedDepartureTime'] < test_frame['RecordedAtTime'])] # CORRECT ANSWER, TO BE REMOVED

### Delay

Study the Delay column and decide on a reasonable range. 

    A large delay might indicate engine trouble or other abnormalities 
    Is it normal for a bus in service to be way ahead of schedule?

Remember that a _positive_ 'Delay' is the number of seconds _behind_ schedule. I.e 

    A bus with delay=600 is 10 minutes BEHIND schedule
    A bus with delay=-600 is 10 minutes AHEAD schedule

In [ ]:
test_frame = test_frame[test_frame['Delay'] > -10000]
test_frame = test_frame[test_frame['Delay'] < 10000]

frame['Delay'].describe()


test_frame = test_frame[test_frame['Delay'] > -1000] # CORRECT ANSWER, TO BE REMOVED
test_frame = test_frame[test_frame['Delay'] < 1000] # CORRECT ANSWER, TO BE REMOVED

### Position

All bus routes should be in the [Rogaland area](https://www.google.com/maps/place/Rogaland/@58.9350028,5.2741278,9z/data=!3m1!4b1!4m5!3m4!1s0x463a353f2adcd70b:0xe0061cba0b0cc0bc!8m2!3d59.1489544!4d6.0143432), but sometimes the GPS freaks out..

In [ ]:
test_frame = test_frame[test_frame['Latitude'] < 61] # CORRECT ANSWER, TO BE REMOVED
test_frame = test_frame[test_frame['Longitude'] < 7.3] # CORRECT ANSWER, TO BE REMOVED
test_frame = test_frame[test_frame['Longitude'] > 4.7] # CORRECT ANSWER, TO BE REMOVED

test_frame = test_frame[test_frame['Latitude'] > 58] 
frame[['Longitude','Latitude']].describe(include='all')

In [ ]:
cleaner.remove_outliers(test_frame)

In [ ]:
frame = test_frame

# Machine learning example

In [ ]:
def cat(x):
    if x <-1:
        return "Ahead of schedule"
    if x > 200:
        return "Very Delayed"
    if x > 100:
        return "Delayed"
    return "On schedule"